# Kaggle Competition Stage 1: Submission 1 - CNN

# Connect GoogleDrive

In [0]:
# install PyDrive, only run one time
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Get authorize, only need in first time you run the code
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# Looking for ID
file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
file_list = drive.ListFile({'q': "'1p3moyb19csZ3O5aQgCX2jT84ISFL3Cd0' in parents and trashed=false"}).GetList()

In [0]:
# Google Drive File ID
listed = drive.ListFile({'q': "title contains 'stage_1_train_labels.csv' and '1p3moyb19csZ3O5aQgCX2jT84ISFL3Cd0' in parents"}).GetList()
for file in listed:
  print('title {}, id {}'.format(file['title'], file['id']))
  
listed = drive.ListFile({'q': "title contains 'stage_1_train_images.zip' and '1p3moyb19csZ3O5aQgCX2jT84ISFL3Cd0' in parents"}).GetList()
for file in listed:
  print('title {}, id {}'.format(file['title'], file['id']))
  
listed = drive.ListFile({'q': "title contains 'stage_1_test_images.zip' and '1p3moyb19csZ3O5aQgCX2jT84ISFL3Cd0' in parents"}).GetList()
for file in listed:
  print('title {}, id {}'.format(file['title'], file['id']))

title stage_1_train_labels.csv, id 1n1EnK1Jq5fe-3rY5t7u0Fe8LvAJPaeW0
title stage_1_train_images.zip, id 1cZtHGtv1S4rw0DBMmGW6AmA0v7Ypwe5V
title stage_1_test_images.zip, id 1j2UYG2sVPReaPhHGZ6sQkzi7wSOAXtG4


In [0]:
# Files
train_zip = drive.CreateFile({'id': '1cZtHGtv1S4rw0DBMmGW6AmA0v7Ypwe5V'}) 
test_zip = drive.CreateFile({'id': '1j2UYG2sVPReaPhHGZ6sQkzi7wSOAXtG4'}) 
train_zip.GetContentFile('stage_1_train_images.zip', "zip")
test_zip.GetContentFile('stage_1_test_images.zip', "zip")
train_labels = drive.CreateFile({'id': '1n1EnK1Jq5fe-3rY5t7u0Fe8LvAJPaeW0'})
train_labels.GetContentFile('stage_1_train_labels.csv', "csv")

In [0]:
# https://blog.csdn.net/ice110956/article/details/26597179
# http://blog.51cto.com/wangwei007/1045577

import zipfile
import os

def un_zip(file_name):
    """unzip zip file"""
    zip_file = zipfile.ZipFile(file_name)
    if os.path.isdir(file_name + "_files"):
        pass
    else:
        os.mkdir(file_name + "_files")
    for names in zip_file.namelist():
        zip_file.extract(names,file_name + "_files/")
    zip_file.close()

un_zip('stage_1_train_images.zip')
un_zip('stage_1_test_images.zip')

# CNN Model


This follow the kaggle tutorial:
 https://www.kaggle.com/jonnedtc/cnn-segmentation-connected-components 

Approach
1.   Firstly a convolutional neural network is used to segment the image, using the bounding boxes directly as a mask.
2.   Secondly connected components is used to separate multiple areas of predicted pneumonia.
3.   Finally a bounding box is simply drawn around every connected component.

Network
*    The network consists of a number of residual blocks with convolutions and downsampling blocks with max pooling.
*    At the end of the network a single upsampling layer converts the output to the same shape as the input.


As the input to the network is 256 by 256 (instead of the original 1024 by 1024) and the network downsamples a number of times without any meaningful upsampling (the final upsampling)

In [0]:
# Install and Import
!pip install pydicom # manually install pydicom to Google Colab 
import os
import csv
import random
import pydicom
import numpy as np
import pandas as pd
import zipfile
from skimage import measure
from skimage.transform import resize

import tensorflow as tf
from tensorflow import keras

from matplotlib import pyplot as plt
import matplotlib.patches as patches

In [0]:
import os
download_path = os.path.expanduser('~/data')
try:
  os.makedirs(download_path)
except FileExistsError:
  pass
from zipfile import ZipFile
train_images = ZipFile('stage_1_train_images.zip', 'r')

#train_images.extractall(os.path.join(download_path, 'stage_1_train_images.zip'))
#train_images.close()

In [0]:
# empty dictionary
pneumonia_locations = {}

# load table
with open('stage_1_train_labels.csv', mode='r') as infile:
    # open reader
    reader = csv.reader(infile)
    # skip header
    next(reader, None)
    # loop through rows
    for rows in reader:
        # retrieve information
        filename = rows[0]
        location = rows[1:5]
        pneumonia = rows[5]
        # if row contains pneumonia add label to dictionary
        # which contains a list of pneumonia locations per filename
        if pneumonia == '1':
            # convert string to float to int
            location = [int(float(i)) for i in location]
            # save pneumonia location in dictionary
            if filename in pneumonia_locations:
                pneumonia_locations[filename].append(location)
            else:
                pneumonia_locations[filename] = [location]

In [0]:
# load and shuffle filenames
folder = './stage_1_train_images.zip_files'
filenames = os.listdir(folder)
random.shuffle(filenames)

# split into train and validation filenames
n_valid_samples = 2560
train_filenames = filenames[n_valid_samples:]
valid_filenames = filenames[:n_valid_samples]
print('n train samples', len(train_filenames))
print('n valid samples', len(valid_filenames))
n_train_samples = len(filenames) - n_valid_samples

n train samples 23124
n valid samples 2560


# Data Generator

The dataset is too large to fit into memory, so we need to create a generator that loads data on the fly.


*   The generator takes in some filenames, batch_size and other parameters.
*   The generator outputs a random batch of numpy images and numpy masks.



In [0]:
class generator(keras.utils.Sequence):
    
    def __init__(self, folder, filenames, pneumonia_locations=None, batch_size=32, image_size=256, shuffle=True, augment=False, predict=False):
        self.folder = folder
        self.filenames = filenames
        self.pneumonia_locations = pneumonia_locations
        self.batch_size = batch_size
        self.image_size = image_size
        self.shuffle = shuffle
        self.augment = augment
        self.predict = predict
        self.on_epoch_end()
        
    def __load__(self, filename):
        # load dicom file as numpy array
        img = pydicom.dcmread(os.path.join(self.folder, filename)).pixel_array
        # create empty mask
        msk = np.zeros(img.shape)
        # get filename without extension
        filename = filename.split('.')[0]
        # if image contains pneumonia
        if filename in self.pneumonia_locations:
            # loop through pneumonia
            for location in self.pneumonia_locations[filename]:
                # add 1's at the location of the pneumonia
                x, y, w, h = location
                msk[y:y+h, x:x+w] = 1
        # resize both image and mask
        img = resize(img, (self.image_size, self.image_size), mode='reflect')
        msk = resize(msk, (self.image_size, self.image_size), mode='reflect') > 0.5
        # if augment then horizontal flip half the time
        if self.augment and random.random() > 0.5:
            img = np.fliplr(img)
            msk = np.fliplr(msk)
        # add trailing channel dimension
        img = np.expand_dims(img, -1)
        msk = np.expand_dims(msk, -1)
        return img, msk
    
    def __loadpredict__(self, filename):
        # load dicom file as numpy array
        img = pydicom.dcmread(os.path.join(self.folder, filename)).pixel_array
        # resize image
        img = resize(img, (self.image_size, self.image_size), mode='reflect')
        # add trailing channel dimension
        img = np.expand_dims(img, -1)
        return img
        
    def __getitem__(self, index):
        # select batch
        filenames = self.filenames[index*self.batch_size:(index+1)*self.batch_size]
        # predict mode: return images and filenames
        if self.predict:
            # load files
            imgs = [self.__loadpredict__(filename) for filename in filenames]
            # create numpy batch
            imgs = np.array(imgs)
            return imgs, filenames
        # train mode: return images and masks
        else:
            # load files
            items = [self.__load__(filename) for filename in filenames]
            # unzip images and masks
            imgs, msks = zip(*items)
            # create numpy batch
            imgs = np.array(imgs)
            msks = np.array(msks)
            return imgs, msks
        
    def on_epoch_end(self):
        if self.shuffle:
            random.shuffle(self.filenames)
        
    def __len__(self):
        if self.predict:
            # return everything
            return int(np.ceil(len(self.filenames) / self.batch_size))
        else:
            # return full batches only
            return int(len(self.filenames) / self.batch_size)

In [0]:
## Network


def create_downsample(channels, inputs):
    x = keras.layers.BatchNormalization(momentum=0.9)(inputs)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(channels, 1, padding='same', use_bias=False)(x)
    x = keras.layers.MaxPool2D(2)(x)
    return x

def create_resblock(channels, inputs):
    x = keras.layers.BatchNormalization(momentum=0.9)(inputs)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(channels, 3, padding='same', use_bias=False)(x)
    x = keras.layers.BatchNormalization(momentum=0.9)(x)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(channels, 3, padding='same', use_bias=False)(x)
    return keras.layers.add([x, inputs])

def create_network(input_size, channels, n_blocks=2, depth=4):
    # input
    inputs = keras.Input(shape=(input_size, input_size, 1))
    x = keras.layers.Conv2D(channels, 3, padding='same', use_bias=False)(inputs)
    # residual blocks
    for d in range(depth):
        channels = channels * 2
        x = create_downsample(channels, x)
        for b in range(n_blocks):
            x = create_resblock(channels, x)
    # output
    x = keras.layers.BatchNormalization(momentum=0.9)(x)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(1, 1, activation='sigmoid')(x)
    outputs = keras.layers.UpSampling2D(2**depth)(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [0]:
# define iou or jaccard loss function
def iou_loss(y_true, y_pred):
    y_true = tf.reshape(y_true, [-1])
    y_pred = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true * y_pred)
    score = (intersection + 1.) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection + 1.)
    return 1 - score

# combine bce loss and iou loss
def iou_bce_loss(y_true, y_pred):
    return 0.5 * keras.losses.binary_crossentropy(y_true, y_pred) + 0.5 * iou_loss(y_true, y_pred)

# mean iou as a metric
def mean_iou(y_true, y_pred):
    y_pred = tf.round(y_pred)
    intersect = tf.reduce_sum(y_true * y_pred, axis=[1, 2, 3])
    union = tf.reduce_sum(y_true, axis=[1, 2, 3]) + tf.reduce_sum(y_pred, axis=[1, 2, 3])
    smooth = tf.ones(tf.shape(intersect))
    return tf.reduce_mean((intersect + smooth) / (union - intersect + smooth))

# create network and compiler
model = create_network(input_size=256, channels=32, n_blocks=2, depth=4)
model.compile(optimizer='adam',
              loss=iou_bce_loss,
              metrics=['accuracy', mean_iou])

# cosine learning rate annealing
def cosine_annealing(x):
    lr = 0.001
    epochs = 25
    return lr*(np.cos(np.pi*x/epochs)+1.)/2
learning_rate = tf.keras.callbacks.LearningRateScheduler(cosine_annealing)

# create train and validation generators
folder = './stage_1_train_images.zip_files'
train_gen = generator(folder, train_filenames, pneumonia_locations, batch_size=32, image_size=256, shuffle=True, augment=True, predict=False)
valid_gen = generator(folder, valid_filenames, pneumonia_locations, batch_size=32, image_size=256, shuffle=False, predict=False)

history = model.fit_generator(train_gen, validation_data=valid_gen, callbacks=[learning_rate], epochs=25, workers=4, use_multiprocessing=True)

Epoch 1/25
722/722 [==============================] - 1659s 2s/step - loss: 15.6339 - acc: 0.9621 - mean_iou: 0.6169 - val_loss: 15.6428 - val_acc: 0.9639 - val_mean_iou: 0.6197
Epoch 2/25
722/722 [==============================] - 1080s 1s/step - loss: 14.4527 - acc: 0.9665 - mean_iou: 0.6697 - val_loss: 14.3044 - val_acc: 0.9610 - val_mean_iou: 0.6323
Epoch 3/25
722/722 [==============================] - 1075s 1s/step - loss: 13.8554 - acc: 0.9682 - mean_iou: 0.6860 - val_loss: 15.3587 - val_acc: 0.9384 - val_mean_iou: 0.5188
Epoch 4/25
722/722 [==============================] - 1082s 1s/step - loss: 13.5442 - acc: 0.9696 - mean_iou: 0.7011 - val_loss: 14.4012 - val_acc: 0.9714 - val_mean_iou: 0.7429
Epoch 5/25
722/722 [==============================] - 1089s 2s/step - loss: 13.3252 - acc: 0.9705 - mean_iou: 0.7087 - val_loss: 17.3704 - val_acc: 0.9724 - val_mean_iou: 0.7762
Epoch 6/25
722/722 [==============================] - 1083s 1s/step - loss: 13.0937 - acc: 0.9711 - mean_iou: 

In [0]:
# plt.figure(figsize=(12,4))
# plt.subplot(131)
# plt.plot(history.epoch, history.history["loss"], label="Train loss")
# plt.plot(history.epoch, history.history["val_loss"], label="Valid loss")
# plt.legend()
# plt.subplot(132)
# plt.plot(history.epoch, history.history["acc"], label="Train accuracy")
# plt.plot(history.epoch, history.history["val_acc"], label="Valid accuracy")
# plt.legend()
# plt.subplot(133)
# plt.plot(history.epoch, history.history["mean_iou"], label="Train iou")
# plt.plot(history.epoch, history.history["val_mean_iou"], label="Valid iou")
# plt.legend()
# plt.show()

In [0]:
# load and shuffle filenames
folder = './stage_1_test_images.zip_files'
test_filenames = os.listdir(folder)
print('n test samples:', len(test_filenames))

# create test generator with predict flag set to True
test_gen = generator(folder, test_filenames, None, batch_size=32, image_size=256, shuffle=False, predict=True)

# create submission dictionary
submission_dict = {}
# loop through testset
for imgs, filenames in test_gen:
    # predict batch of images
    preds = model.predict(imgs)
    # loop through batch
    for pred, filename in zip(preds, filenames):
        # resize predicted mask
        pred = resize(pred, (1024, 1024), mode='reflect')
        # threshold predicted mask
        comp = pred[:, :, 0] > 0.5
        # apply connected components
        comp = measure.label(comp)
        # apply bounding boxes
        predictionString = ''
        for region in measure.regionprops(comp):
            # retrieve x, y, height and width
            y, x, y2, x2 = region.bbox
            height = y2 - y
            width = x2 - x
            # proxy for confidence score
            conf = np.mean(pred[y:y+height, x:x+width])
            # add to predictionString
            predictionString += str(conf) + ' ' + str(x) + ' ' + str(y) + ' ' + str(width) + ' ' + str(height) + ' '
        # add filename and predictionString to dictionary
        filename = filename.split('.')[0]
        submission_dict[filename] = predictionString
    # stop if we've got them all
    if len(submission_dict) >= len(test_filenames):
        break

n test samples: 1000


In [0]:
# save dictionary as csv file
sub = pd.DataFrame.from_dict(submission_dict,orient='index')
sub.index.names = ['patientId']
sub.columns = ['PredictionString']
sub.to_csv('1.csv')

# New Section